In [ ]:
# Input info
se_cells_meta_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/knn/kparam_30/gff_A2_black/annotation_clones/se_cells_meta_labels.tsv"
#mt_as_clones_dir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/donor_0_thresh_results.tsv"
mt_as_clones_dir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh"

outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/clonal_shifts/"


atac_col = "cluster_labels"
clone_col = "Variants"

af_t = 0.1
oth_af_t = 0.1
cov_t = 10
oth_cov_t = 10
ncells = 10
oth_ncells = 0.25 
mean_pos_cov = 0

# config
N_DONORS = 2
input_cond = "Input"

# params
min_clone_size = 10
p_thresh = 0.1 


#conds_sep = False

counts_f = ""


In [ ]:
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from src import clonal_shifts as cs


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cells_dir=join(mt_as_clones_dir, "cells")
labels_df = pd.read_csv(se_cells_meta_f,sep="\t").set_index("ID")
print(labels_df.shape)

labels_df = labels_df[~(labels_df["donor"]=='None')]
print(labels_df.shape)
labels_df.head()
def fill_mt_bin(curr_pos, curr_cells):
    print(curr_pos.name)
    curr_pos.loc[curr_cells[curr_pos.name]["other_cells"]] = 0
    curr_pos.loc[curr_cells[curr_pos.name]["clone_cells"]] = 1
    return curr_pos
bin_d = {}
for d in range(N_DONORS): 
    curr_f = join(cells_dir, f"don.{d}_af.{af_t}_othaf.{oth_af_t}_cov.{cov_t}_othcov.{oth_cov_t}.p")
    curr_cells = pickle.load(open(curr_f,'rb'))
    params_results = pd.read_csv(join(mt_as_clones_dir, f"donor_{d}_thresh_results.tsv"), sep="\t")    


    curr_p = params_results.loc[(params_results["af"]==af_t) &
                       (params_results["oth_af"]==oth_af_t) &
                       (params_results["cov"]==cov_t) &
                       (params_results["oth_cov"]==oth_cov_t) &
                       (params_results["ncells"]==ncells) &
                       (params_results["oth_ncells"]==oth_ncells) &
                       (params_results["mean_cov"]==mean_pos_cov)]
    assert(len(curr_p)==1)
    
    ## Construct a binary cell-by-variant matrix for the kept variants. 
    ## Can add N/A if not in oth cells
    curr_vars = curr_p.iloc[0]["Variants"].split(";")

    all_cells = set()
    for x in curr_cells:
        all_cells = all_cells.union(set(curr_cells[x]["clone_cells"])) 
        all_cells = all_cells.union(set(curr_cells[x]["other_cells"])) 

    mt_bin = pd.DataFrame(index = all_cells, columns=curr_vars)
    bin_d[d] = mt_bin.apply(fill_mt_bin, curr_cells=curr_cells,axis=0)


In [ ]:
def get_mt_ncells(curr_df, bin_d):
    lab, don = curr_df.name
    curr_bin = bin_d[int(don)]
    out = curr_bin.loc[curr_bin.index.isin(curr_df.index)].sum(axis=0).fillna(0)
    out.name = "count"
    return out #curr_bin.loc[curr_df.index].sum(axis=0)
    
lin_mt_ncells = labels_df.groupby(["cluster_labels", "donor"]).apply(get_mt_ncells, bin_d)
lin_mt_ncells = lin_mt_ncells.reset_index().rename({"level_2":"Variants"}, axis=1)

# convert back to raw cell count from log2
#lin_mt_ncells["count"] = (np.ceil((2**lin_mt_ncells["count"])-1)).astype(int)

In [ ]:
lin_mt_ncells

In [ ]:
sep_dir = join(outdir,"sepDonors")
if not exists(sep_dir):
    os.mkdir(sep_dir)
donor_out = {}
for d in np.arange(N_DONORS):
    donor_out[d] = join(sep_dir, f"donor{d}")
    if not exists(donor_out[d]):
        os.mkdir(donor_out[d])
    if not exists(join(donor_out[d], "shuffle")):
        os.mkdir(join(donor_out[d], "shuffle"))

In [ ]:
#for d, val in cells_meta.groupby("donor"):
#for curr_don, don_lin_mt_ncells in lin_mt_ncells.groupby("donor"):

for d, val in lin_mt_ncells.groupby("donor"):
    #curr_groups = val.groupby([atac_col, clone_col]).size().reset_index().rename({0:"count"}, axis=1)
    curr_groups = val
    curr_sizes = val.groupby(clone_col).size().sort_values(ascending=False)
    curr_name_cond_size = val.groupby([clone_col]).size()
    curr_name_cond_size = curr_name_cond_size[curr_name_cond_size>min_clone_size]
    curr_clones_filt = curr_name_cond_size.index 
    curr_sizes = curr_sizes.loc[curr_clones_filt].sort_values(ascending=False)
    
    curr_clones = np.unique(curr_groups[clone_col])
    curr_clone_map = {x:ind for ind,x in enumerate(curr_clones)}
    print(curr_clones)
    curr_atac_cl = np.unique(curr_groups[atac_col])
    print(curr_atac_cl)

    output_df, bh_enrichment_df = cs.pipeline_groups_hypergeo(curr_groups, curr_clones, curr_atac_cl, curr_sizes,
                                                             p_thresh=p_thresh, atac_col=atac_col, 
                                                              clone_col=clone_col)
    #cs.pipeline_groups_hypergeo(curr_groups, curr_clones, curr_atac_cl, curr_sizes)

    curr_don_out = donor_out[int(d)]
    bh_enrichment_df.to_csv(join(curr_don_out, "hypergeo_input_padjusted.csv"))
    output_df.to_csv(join(curr_don_out, "hypergeo_input_padjusted_sigOnly.csv"))
    
    if output_df.shape[0] == 0:   
        g = sns.clustermap(-np.log10(bh_enrichment_df.fillna(1)), 
                       row_cluster=False)
        g.fig.suptitle("No groups were significant")
    else:
        g = sns.clustermap(-np.log10(bh_enrichment_df.loc[output_df.index].fillna(1)), 
                       row_cluster=False)
    g.ax_heatmap.set(xlabel="Cluster ID")
    g.ax_cbar.set(title="-log10 p-value")
    plt.savefig(join(curr_don_out, "hypergeo_input_padjusted_sigOnly.png"))
    
    
    init_bh_enrichment = cs.create_enrichment(curr_groups, atac_col, clone_col, p_thresh,
                                              clones=curr_clones, atac_cl=curr_atac_cl)
    
    shuffle = cs.shuffle_hypergeo(curr_groups, atac_col, clone_col, p_thresh, curr_clones, curr_atac_cl, 
                                  n_shuffle=1000, to_parallel=True, n_cpus=24)

    results_df, out_d = cs.get_out(shuffle, curr_clones, init_bh_enrichment, p_thresh, 
                                                          curr_clone_map, atac_col, 
                                                          outdir=join(curr_don_out, "shuffle"))
    
    
#     # plot just the counts
#     curr_groups["log2_count"] = np.log2(curr_groups["count"]+1)
#     g = sns.clustermap(curr_groups.pivot(index=atac_col,columns=clone_col, values="log2_count").fillna(0))
#     plt.gca().set_title("log2 ncells")
#     plt.savefig(join(curr_don_out, "ncells.png"))
    

In [ ]:
curr_clones

In [ ]:
out_d

In [ ]:
results_df

In [ ]:
#cs.create_enrichment(curr_groups, atac_col, clone_col, p_thresh, clones=curr_clones, atac_cl=curr_atac_cl)
#     clones, atac_cl = get_groups(curr_groupsr, curr_clones, curr_atac_cl, clone_col,
#                                  atac_col)

In [ ]:
for curr_don, don_lin_mt_ncells in lin_mt_ncells.groupby("donor"):
    curr_don_mt = don_lin_mt_ncells.set_index(["cluster_labels","donor"])
    #lin_mt_ncells_w = curr_don_mt.reset_index().melt(id_vars=["cluster_labels", "donor"], var_name="Variant", value_name="ncells")
    don_lin_mt_ncells_df = don_lin_mt_ncells.pivot(index="cluster_labels", columns="Variants", values="ncells" )
    sns.clustermap(don_lin_mt_ncells_df.fillna(0))

In [ ]:
don_lin_mt_ncells

## Load cells_meta

In [ ]:
cells_meta = pd.read_csv(se_cells_meta_f, sep="\t")
cells_meta = cells_meta.loc[~(cells_meta["name"]=="None")]

if not "cluster_labels" in cells_meta.columns.values:
    cells_meta["cluster_labels"] = cells_meta["seurat_clusters"]


## Map the new group to cells_meta

In [ ]:
cells_meta[clone_col] = cells_meta.apply(lambda x: barcodes_in[int(x["donor"])].loc[x["name"], clone_col] , axis=1)
cells_meta

## Filter for input only

In [ ]:
cells_meta = cells_meta.loc[cells_meta["condition"]==input_cond]
cells_meta.head()